## Set Up Environment

In [ ]:
# Cell 0

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20210124194550.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [ ]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
# Cell 3

wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)
# wallet = btest_wallet

In [ ]:
wallet.get_address()

In [ ]:
icon_service.get_balance(wallet.get_address()) / 10**18

### Send ICX to Balanced test wallet

In [ ]:
# Cell 4
# Only necessary if running locally or on the private tbears server.

test_wallet = KeyWallet.load("./keystores/keystore_test1.json", "test1_Account")
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(1000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, test_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

### Deployed to testnet

In [ ]:
# Cell 5
# The following addresses are those deployed to the testnet at the start of the audit.

# Uncomment these lines and comment out the definition below to load contracts from a saved file.
# with open(contracts_file_custom, 'rb') as f:
#     contracts = pkl.load(f)

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxcd9ae763fabcf6c3c9a475f727264e9cfffdfaf2'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx52aab6b1dd316396e684573464af36c634059721'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx02bba98bf78c91329c6958c61ceed1e7bad3b68a'},
 'reserve': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cx418f4a6392b88568cb9587c6ca638b358f8ca728'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx65fb9d6c9bafdc9cdbe6bf258fd31b90f2f4ab8d'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx291dacbb875a94b364194a5febaac4e6318681f7'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx4179247372b8ce4c9edd8013dd3918852bd0c7fa'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx5b0d6a5b244e9254ca05592389672c5185d9f5f8'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cx699694fa8dbb76d904d33b63f90dee7cf0440e71'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxd89295b894b72b3b5966848f6fe2bbb997f99e71'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx6fda38cfc5c828763306ad7fb4225a8062bd0aee'}}

In [ ]:
# New testnet deployment, 3/02/2021

# staking = 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c'
# sicx = 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281'

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxc3a8dd98d922e2a1e4a6e9c836c410a1d564ad7e'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxc337abaf1b24e13e827644501716ebaaab9493f2'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx0c58366ebf1b05fa917754e4e5d3873c016d39b1'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cxf9a0c7866fc95e34ee041fb38fa568520719c1cf'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx67da40d0c49fa340954c6945c83860b52a581653'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cxdf85ae1d175a8978955a0149914ad697d78d18fe'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx9806e13bad38e4fca402eb2fde9779d303fc428a'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxcdae80da2964665c5b2480477a44b9646511d7ee'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cx0399a75f88323f13daea97f114440f14fd551494'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxdfa188a9ef06d9e6a5118b9c73c3fac1567bc889'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cxa2dd90f9b85a22c3514e21f309c3e52e8eec02af'}}

In [ ]:
# Private tbears server - with 1 hour day.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx354c0ce44e543ebce9478bf0de3621c95473fc90'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx0a478572e443bbfd66a35d124f63a2db9b637483'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxe6074249f58898af2d6d00859ecb54e3f658ebe5'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxeb7eb8d592ce76d0fcf1a3edd56b50ad2f1f72c5'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx355d0e49525a681723ac0df9347189b5bb7814c8'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxc60ef756b875786c9f820313ac4015e45726b47d'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxbc2752abceba2bf179c1bfe0bc3f88caedcf2bd9'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx0c6bb52e5e34ad8848de911be8ae52be38992363'}}

In [ ]:
# 03/04/2021 Testnet

{"loans": "cxc3a8dd98d922e2a1e4a6e9c836c410a1d564ad7e",
 "staking": "cxc337abaf1b24e13e827644501716ebaaab9493f2",
 "dividends": "cx0c58366ebf1b05fa917754e4e5d3873c016d39b1",
 "reserve": "cxf9a0c7866fc95e34ee041fb38fa568520719c1cf",
 "rewards": "cx67da40d0c49fa340954c6945c83860b52a581653",
 "dex": "cxdf85ae1d175a8978955a0149914ad697d78d18fe",
 "governance": "cx9806e13bad38e4fca402eb2fde9779d303fc428a",
 "dummy_oracle": "cx61a36e5d10412e03c907a507d1e8c6c3856d9964",
 "sicx": "cxcdae80da2964665c5b2480477a44b9646511d7ee",
 "icd": "cx0399a75f88323f13daea97f114440f14fd551494",
 "bal": "cxdfa188a9ef06d9e6a5118b9c73c3fac1567bc889",
 "bwt": "cxa2dd90f9b85a22c3514e21f309c3e52e8eec02af"}

In [ ]:
{"loans": "cx1bdfc395e3c9ae3ea4a4efa7287367c59c3e24fb",
 "staking": "cxdc140c1d6a4ba79037b76adff8b94970cdde2f25",
 "dividends": "cxdebb4f8f9e3d6c705acbfc878e748250daa1d687",
 "reserve": "cx17d4ac46533ac046ed0d96637b4e1676ceed3f94",
 "rewards": "cxe8bdae7e809fd1fa6447b0246e5f47e5afc74f13",
 "dex": "cxb07a58e85729ce51ff89dda2cade64f40c679d01",
 "governance": "cx23258bfc634b3d6f2e87cb68c8dcc55342c2c53b",
 "dummy_oracle": "cx355d0e49525a681723ac0df9347189b5bb7814c8",
 "sicx": "cxc60ef756b875786c9f820313ac4015e45726b47d",
 "icd": "cxb5e629e5ffc84f55835505030c996e5f13865f6a",
 "bal": "cx9e119a716f99360c516228240ab14ae5dd314da2",
 "bwt": "cxf56ba1e3e3a8c198e8acf88cd450faf2facfdc06"}

In [ ]:
# Cell 6

deploy = list(contracts.keys())[:]
deploy.remove('dummy_oracle')
# deploy.remove('staking')
# deploy.remove('sicx')
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'

contracts

## Deploy All SCOREs

In [ ]:
# Cell 7
# prints out contracts dictionary with all new contract addresses

step_limit = 4000100000

for score in deploy:
    loc = contracts[score]
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

#     step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
    print(score)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('\n')

contracts

In [ ]:
# Cell 8
# saves contracts dictionary to file

# now = datetime.utcnow()
# filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
# print(f'saving contracts list as {filename}')
# with open(filename, 'wb') as f:
#     pkl.dump(contracts, f)

In [ ]:
# Cell 9
# Prints out dictionary of SCORE addresses for use in testing UI

def get_scores_json(contracts):
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

print(get_scores_json(contracts))

### Deploy a single SCORE

In [ ]:
# Cell 10

# Deploy a single SCORE
contract = 'staking'
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

step_limit = 10000000000


signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
contracts[contract]["SCORE"] = res.get('scoreAddress', '')
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

## Update a SCORE

In [ ]:
# Cell 11

contract = 'dex'
update = contracts[contract]['SCORE']
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

# step_limit = icon_service.estimate_step(deploy_transaction) + 100000
step_limit = 3000000000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

## Set parameters

In [ ]:
# Cell 12

addresses = {'loans': contracts['loans']['SCORE'],
             'dex': contracts['dex']['SCORE'],
             'staking': contracts['staking']['SCORE'],
             'rewards': contracts['rewards']['SCORE'],
             'reserve': contracts['reserve']['SCORE'],
             'dividends': contracts['dividends']['SCORE'],
             'oracle': contracts['dummy_oracle']['SCORE'],
             'sicx': contracts['sicx']['SCORE'],
             'icd': contracts['icd']['SCORE'],
             'bal': contracts['bal']['SCORE'],
             'bwt': contracts['bwt']['SCORE']
            }

settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'sicx', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'staking', 'method': 'setSicxAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'icd', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'bal', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'bwt', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'dex', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'reserve', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'rewards', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'dividends', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'loans', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'governance', 'method': 'setAddresses', 'params':{'_addresses': addresses}},
            {'contract': 'governance', 'method': 'launchBalanced', 'params':{}}]


In [ ]:
# Cell 13

results = {}
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    results[f'{sett["contract"]}|{sett["method"]}|{sett["params"]}'] = res
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

## Run through read-only methods on all SCOREs

In [ ]:
# Cell 14

ro_methods = {"loans": ["name", "getAssetTokens", "getCollateralTokens", "getTotalCollateral", "getAvailableAssets", "assetCount", "getSnapshot"],
              "staking": ["name", "getSicxAddress", "getTodayRate"],
              "dividends": ["name", "getBalances"],
              "reserve": ["name", "getBalances"],
              "rewards": ["name", "getDataSourceNames"],
              "governance": ["name", "getAddresses"],
              "sicx": ["name", "symbol", "decimals", "totalSupply", "getAdmin", "lastPriceInLoop"],
              "icd": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "bal": ["name", "symbol", "decimals", "totalSupply", "getAdmin"]
              }

In [ ]:
# Cell 15
# Runs through the readonly methods defined above. These can also be viewed using the testing UI

results = {}
for score in ro_methods.keys():
    print()
    print(f'SCORE: {score}')
    print(f'Address: {contracts[score]["SCORE"]}')
    results[score] = {}
    for method in ro_methods[score]:
        call = CallBuilder().from_(wallet.get_address())\
                            .to(contracts[score]['SCORE'])\
                            .method(method)\
                            .build()
        result = icon_service.call(call)
        if method in ['totalSupply', 'decimals', 'get_total_collateral', 'get_rate']:
            result = int(result, 0)
            if method in ['totalSupply', 'get_total_collateral', 'get_rate']:
                result = result / ICX
        print(f'{method}: {result}')
        results[score][method] = result

### Get sICX from Staking

In [ ]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params({})\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

## Deposit ICX and Mint ICD Loan

In [ ]:
# Cell 16 - Use tbears test account to send 800 ICX to the addCollateral method on the Loans contract with data instructing the
# _deposit_and_borrow method to mint 50 ICD.
# Check that there is 800 ICX worth of collateral for position 1
# Check that there is a 50 ICD loan for position 1.

params = {'_asset': 'ICD', '_amount': 50 * ICX}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

### Just deposit collateral

In [ ]:
# Cell 17

params = {'_asset': '', '_amount': 0}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(30 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

### Repay Some ICD

In [ ]:
# Cell 18
params = {"method": "_repay_loan", "params": {}}
data = json.dumps(params).encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 5 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

### Withdraw Some Collateral

In [ ]:
# Cell 19

params = {'_value': 25 * ICX}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("withdrawCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

### Add collateral directly

In [ ]:
params = {"_sender": "hxe7af5fcfd8dfc67530a01a0e403882687528dfcb", "_asset": "", "_amount": 0}
data = json.dumps({"method": "_deposit_and_borrow", "params": params}).encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 20 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

### Test Liquidation

In [ ]:
btest_wallet.get_address()

In [ ]:
# Cell 20
# 1. Deposit collateral to fresh wallet

params = {'_asset': '', '_amount': 0}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(782769 * ICX // 1000)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 21
# 2. Check the account standing after depositing collateral - Should be 'No Debt', and
#    holding '0x2a6f1a22364bbe8000' worth of sICX.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

#### 3. Call the test method to add a loan of ICD to the Balanced test wallet
The test method will mint and add the ICD. Adds an undercollateralized amount of ICD, 200 ICD.

In [ ]:
# Cell 22
# 3a. Mints an amount of ICD to the test address without checking collateralization ratio
#     With the above collateral deposit it will put the position in a standing of Liquidation.

params = {}
transaction = CallTransactionBuilder()\
    .from_(address)\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(100000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_bad_test_position")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 23
# 4. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' ICD and have standing of 'Liquidate'.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
# Cell 24
# 5. Liquidate the account position.

params = {'_owner': btest_wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("liquidate")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('checkDebts')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
# Cell 25
# 6. Check the standing of the account after liquidation. Should now have zero balance for sICX and ICD.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
# Create a set of wallets

# wallets = []
# for i in range(200):
#     wallets.append(KeyWallet.create())
# with open("../wallets.pkl", 'wb') as f:
#     pkl.dump(wallets, f)

In [ ]:
with open("../wallets.pkl", 'rb') as f:
    wallets = pkl.load(f)

In [ ]:
for i in range(200):
    print(f'{i:3d}: {icon_service.get_balance(wallets[i].get_address()) / 10**18: 9.3f}')

In [ ]:
# 10000 ICX into each wallet. Uncomment if needed. These wallets are already more than well stocked enough on the tbears server.

# for i in range(10):
#     transaction = TransactionBuilder()\
#         .from_(wallet.get_address())\
#         .to(wallets[i].get_address())\
#         .value(20 * ICX)\
#         .step_limit(1000000) \
#         .nid(NID) \
#         .nonce(2) \
#         .version(3) \
#         .build()
#     signed_transaction = SignedTransaction(transaction, wallet)
#     tx_hash = icon_service.send_transaction(signed_transaction)
# tx_hash

In [ ]:
# Mint some ICD to first 20 wallets.

for i in range(20):
    params = {'_asset': 'ICD', '_amount': 50 * ICX}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(800 * ICX)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("addCollateral")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# redeem 20 ICD from each of 20 wallets that do not have positions on Balanced. This will use up all
# of the liquidation pool and require some replay events to be recorded.

for i in range(20):
    params = {'_to': wallets[i+100].get_address(), '_value': 30 * ICX}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['icd']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)
    params = {"method": "_retire_asset", "params": {}}
    data = json.dumps(params).encode("utf-8")
    params = {'_to': contracts['loans']['SCORE'], '_value': 50 * ICX, '_data': data}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i+100].get_address())\
        .to(contracts['icd']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i+100])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Gets the total number of borrowers in the system and iterates over them by index to update each one.

params = {}
call1 = CallBuilder().from_(wallet.get_address())\
                     .to(contracts['loans']['SCORE'])\
                     .method("borrowerCount")\
                     .params(params)\
                     .build()
total_borrowers = int(icon_service.call(call1), 0)
for i in range(total_borrowers):
    params = {'_index': i + 1}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['loans']['SCORE'])\
                        .method("getPositionAddress")\
                        .params(params)\
                        .build()
    params = {'_owner': icon_service.call(call)}
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"ICD\", \"_amount\": 5000000000000000000}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(18 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"ICD\", \"_amount\": 4000000000000000000}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(wallets[2].get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(80 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallets[2])
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Mint some ICD to each wallet

for i in range(80, 110):
    params = {'_asset': 'ICD', '_amount': 50 * ICX}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(600 * ICX)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("addCollateral")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for _ in range(100):
    params = {'_index': 1}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['loans']['SCORE'])\
                        .method("getPositionAddress")\
                        .params(params)\
                        .build()
    params = {'_owner': icon_service.call(call)}
    transaction = CallTransactionBuilder()\
        .from_(wallets[1].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[1])
    tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for i in range(100):
    params = {'_owner': wallets[i].get_address()}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)

    print(i)
    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

In [ ]:
params = {'_holders': [wallet.get_address(), btest_wallet.get_address()]}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
i = 0
{i: [key, result[key]] for i, key in enumerate(result.keys())}

In [ ]:
# Check BALN that has been distributed on the rewards SCORE, and to platform contract addresses.

params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method("borrowerCount")\
                    .params(params)\
                    .build()
borrowerCount = int(icon_service.call(call), 0)

addresses = []
for i in range(1, borrowerCount + 1):
    params = {'_index': i, '_day': -1}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['loans']['SCORE'])\
                        .method("getPositionByIndex")\
                        .params(params)\
                        .build()
    position = icon_service.call(call)
    addresses.append(position['address'])

params = {'_holders': addresses}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
holders = icon_service.call(call)

total_balances = 0
baln_balances = {}
for contract in ['rewards', 'reserve', 'bwt']:
    params = {'_owner': contracts[contract]['SCORE']}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['bal']['SCORE'])\
                        .method("balanceOf")\
                        .params(params)\
                        .build()
    result = int(icon_service.call(call), 0)
    baln_balances[contract] = result / 10**18
    total_balances += result

i = 0
holdings = {i: [key, int(holders[key], 0), int(holders[key], 0) / 10**18] for i, key in enumerate(holders.keys())}
total = 0
for key in holdings:
    total += holdings[key][1]
    print(f'{holdings[key]}')

print(f'Total unclaimed: {total / 10**18}')
print(baln_balances)
print(f'Total BALN: {total_balances / 10**18}')

In [ ]:
params = {}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['rewards']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("claimRewards")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)

print(i)
res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for i in range(0, 200):
    params = {}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['rewards']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("claimRewards")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

    print(i)
    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

In [ ]:
params = {'_name': 'BALNICD', '_snapshot_id': 1, '_limit': 10, '_offset': 0}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method("getDataBatch")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
params = {'_owner': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("updateStanding")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
icon_service.get_balance(wallet.get_address()) / 10**18

In [ ]:
balances = {}
for wallet in wallets[:20]:
    address = wallet.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['bal']['SCORE'])\
                        .method('availableBalanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = result

In [ ]:
balances = {}
for wallet in wallets[:20]:
    address = wallet.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['sicx']['SCORE'])\
                        .method('balanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = result

### DEX SICX Test

Does the following:

1. Sends ICX to the DEX
2. Checks overall ICX balance
3. Checks withdrawal lock
4. Mints some SICX
5. Trades SICX against ICX

In [ ]:
contracts['dex']['SCORE']

In [ ]:
# Cell 26

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 27

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 28

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXWithdrawLock')\
                    .build()
result = icon_service.call(call)
datetime.utcfromtimestamp(int(result, 0) // 10**6).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# Cell 29

params = {'_to': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 30

data = "{\"method\": \"_swap_icx\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(30000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 31

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 32

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 33

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDataBatch')\
                    .params({'_name': 'SICXICX', '_limit': 10, '_snapshot_id': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 34

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
wallet.get_address()

In [ ]:
# Cell 35

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 36

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bal']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 37

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDeposit')\
                    .params({'_user': wallet.get_address(), '_tokenAddress': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 38a

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['bal']['SCORE'], '_quoteToken': contracts['icd']['SCORE'], '_baseValue': 100 * ICX, '_quoteValue': 30990197728645406}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 38b

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['icd']['SCORE'], '_baseValue': 1 * ICX, '_quoteValue': 1 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 39

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("dexPermit")\
    .params({'_pid': 1, '_permission': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 40

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getInversePrice')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**10

In [ ]:
# Cell 41

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 42

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolId')\
                    .params({'_token1Address': contracts['sicx']['SCORE'], '_token2Address': contracts['icd']['SCORE'] }) \
                    .build()
result = icon_service.call(call)
print("pool w/ tokens: " + str(contracts['sicx']['SCORE']) + ' & ' + str(contracts['icd']['SCORE']))
print(result)

In [ ]:
# Cell 43

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolBase')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolQuote')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 44

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':2, '_name': 'SICXICD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 44

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':3, '_name': 'BALNICD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 45

to_token = contracts['sicx']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":10}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': ICX // 100, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 45

to_token = contracts['icd']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': ICX // 100, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 46

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['sicx']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 47

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 48

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('totalSupply')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 49

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("remove")\
    .params({'_pid':2, '_value': 1 * ICX, '_withdraw': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 50

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 30 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 51

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 52

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("cancelSicxicxOrder")\
    .params({}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':3, '_name': 'BALNICD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
to_token = contracts['bal']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')